In [1]:
import nltk
import wikipedia
import string
import math
import numpy as np
import pandas as pd

In [2]:
# nltk.download_shell()

In [3]:
stopwords_list = nltk.corpus.stopwords.words('indonesian')

In [4]:
wikipedia.set_lang('id')

In [5]:
keyword = input("Keyword : ")

Keyword : singa


In [6]:
dataTraining = wikipedia.summary(keyword).lower()

In [7]:
print(dataTraining)

singa (bahasa sanskerta: siṃha, atau nama ilmiahnya panthera leo) adalah spesies hewan dari keluarga felidae atau jenis kucing. singa merupakan hewan yang hidup berkelompok. biasanya terdiri dari seekor jantan dan banyak betina. kelompok ini menjaga daerah kekuasaannya. umur singa antara 10 sampai 15 tahun di alam bebas, tetapi dalam penangkaran memungkinkan lebih dari 20 tahun.
singa betina jauh lebih aktif dalam berburu, sedangkan singa jantan lebih santai bersikap menunggu dan meminta jatah dari hasil buruan para betinanya. singa jantan dipercaya lebih unggul dan perkasa dibandingkan dengan kucing besar lainnya, tetapi kelemahan singa ialah tidak bisa memanjat pohon sebagus kucing-kucing besar lainnya. singa jantan ditumbuhi bulu tebal di sekitar tengkuknya, hal ini lebih menguntungkan untuk melindungi tengkuknya, terutama dalam perkelahian bebas antara kucing besar yang cenderung menerkam tengkuk untuk melumpuhkan musuhnya.
kucing besar lainnya, seperti citah dan macan tutul memili

In [8]:
def text_proses(key):
    
    dataTraining_Selection = [word for word in key if word not in string.punctuation]
    dataTraining_Selection = ''.join(dataTraining_Selection)

    stopwords = [word for word in dataTraining_Selection.split() if word.lower() not in stopwords_list]
    RemoveNya = [word.replace('nya', '') for word in stopwords]
    return RemoveNya

In [9]:
data = text_proses(dataTraining)
dataJoin = " ".join(data)

In [10]:
dataset = pd.read_excel("dataset.xlsx")

In [11]:
dataset

,id,name,index
0,0,kucing,10
1,1,kucing,11
2,2,kucing,12
3,3,kucing,4
4,4,singa,10
5,5,singa,11
6,6,singa,12
7,7,singa,23
8,8,singa,0
9,9,siṃha,1


In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

### Korelasi Antara Dataset dengan DataTesting yang tlah diCleaning

In [13]:
count = CountVectorizer(analyzer=text_proses).fit(dataset['name'].drop_duplicates())
countDataTesting = CountVectorizer(analyzer=text_proses).fit(data)

In [14]:
mess = dataset['name'].values

Jumlah Kesamaan Data

In [15]:
count1 = count.transform([dataJoin])

In [16]:
a = count1.copy()
countCopy = count


labelSet = [countCopy.get_feature_names()[x] for x in a.indices if countCopy.get_feature_names()[x] != keyword]
munculSet = [a.data[x] for x in range(len(a.indices)) if countCopy.get_feature_names()[a.indices[x]] != keyword]

pdSet = pd.DataFrame({'label':labelSet, 'muncul':munculSet}).sort_values(axis=0, by=['muncul', 'label'], ascending=False)

In [17]:
listData = []
listfoundData = []
for a in data:
    if a not in listData:
        listData.append(a)
    else:
        listfoundData.append(a)
        
listfoundData.sort()
listData.sort()
npLabel = []
countLabel = []
for a in listfoundData:
    if a not in npLabel and a != keyword:
        npLabel.append(a)
        countLabel.append((listfoundData.count(a) + 1))


pdTrain = pd.DataFrame({'label':npLabel, 'muncul':countLabel}).sort_values(by=['muncul', 'label'], axis=0, ascending=False)

Hasil Keyword yang muncul berulang pada DataTrain

In [18]:
pdTrain

,label,muncul
11,kucing,4
8,jantan,4
13,tengkuk,3
2,betina,3
12,suka,2
10,keluarga,2
9,jinak,2
7,hewan,2
6,harimau,2
5,film,2


Hasil Dataset x DataTrain

In [19]:
pdSet

,label,muncul
4,kucing,4
3,jantan,4
0,betina,3
2,hewan,2
1,harimau,2
5,siṃha,1


Keyword DataTrain yang terdapat pada DataSet

In [20]:
pdTrainCopy = pdTrain.drop(pdTrain[(pdTrain['muncul'] < 3)].index, axis=0)
pdTrainSelection = pdTrain.drop(pdTrain[(pdTrain['muncul'] > 2)].index, axis=0)
pdTrainSelection.reset_index(drop=True, inplace=True)

dataConcat = pd.concat([pdSet, pdTrainCopy], axis=0)
dataConcat.drop_duplicates(subset='label', inplace=True)
dataConcat.sort_values(by=['muncul'], axis=0, ascending=False, inplace=True)
dataConcat.reset_index(drop=True, inplace=True)
dataConcat

,label,muncul
0,kucing,4
1,jantan,4
2,betina,3
3,tengkuk,3
4,hewan,2
5,harimau,2
6,siṃha,1


In [21]:
for i in dataConcat['label']:
    pdTrainSelection.drop(pdTrainSelection[pdTrainSelection['label'] == i].index, axis=0, inplace=True)

Keyword DataTrain yang tidak terdapat pada DataSet yang akan diseleksi untuk mengetahui tingkat relevan

In [22]:
pdTrainSelection

,label,muncul
0,suka,2
1,keluarga,2
2,jinak,2
5,film,2
6,dunia,2
7,bongo,2
8,bebas,2
9,air,2


In [23]:
def searchRelevanWord(key):
    dataRelevan = wikipedia.summary(key).lower()
    dataKey = text_proses(dataRelevan)
    countKey = 0
    for i in dataKey:
        if i == keyword:
            countKey += 1
    return countKey

In [25]:
dfRelevan = pd.DataFrame({'label': dataConcat['label'], 'jumlah':np.zeros(len(dataConcat['label']))}).sort_values(by=['jumlah'], ascending=False)

for y in range(len(dfRelevan['label'])):
    try:   
        search = searchRelevanWord(dfRelevan.loc[y, 'label'])
        dfRelevan.loc[y, 'jumlah'] = search
    except (wikipedia.DisambiguationError, Exception) as e:
        print('gagal', dfRelevan.loc[y, 'label'])
        dfRelevan.loc[y, 'jumlah'] = -1
        
dfRelevan.sort_values(by=['jumlah'], axis=0, ascending=False, inplace=True)
dfRelevan.reset_index(drop=True, inplace=True)
dfRelevan

,label,jumlah
0,kucing,1.0
1,harimau,1.0
2,jantan,0.0
3,betina,0.0
4,tengkuk,0.0
5,hewan,0.0
6,siṃha,0.0


In [50]:
dfRelevanSeleksi = pd.DataFrame({'label': pdTrainSelection['label'], 'jumlah':np.zeros(len(pdTrainSelection['label']))})
dfRelevanSeleksi.reset_index(drop=True, inplace=True)

for y in range(len(dfRelevanSeleksi['label'])):
    try:   
        search = searchRelevanWord(dfRelevanSeleksi.loc[y, 'label'])
        dfRelevanSeleksi.loc[y, 'jumlah'] = search
    except Exception as e :
        print('gagal', dfRelevanSeleksi.loc[y, 'label'])
        dfRelevanSeleksi.loc[y, 'jumlah'] = -1
        
dfRelevanSeleksi.sort_values(by=['jumlah'], axis=0, ascending=False, inplace=True)
dfRelevanSeleksi.reset_index(drop=True, inplace=True)
dfRelevanSeleksi

C:\Users\LENOVO\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\LENOVO\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


gagal bongo
gagal bebas


,label,jumlah
0,suka,0.0
1,keluarga,0.0
2,jinak,0.0
3,film,0.0
4,dunia,0.0
5,air,0.0
6,bongo,-1.0
7,bebas,-1.0


In [26]:
wikipedia.summary('harimau').lower()

'harimau (nama ilmiah: panthera tigris; disebut juga sebagai macan, meskipun lebih sering digunakan untuk hewan dari spesies panthera pardus beserta subspesiesnya) adalah hewan yang tergolong dalam filum chordata, subfilum vertebrata, kelas mamalia, pemakan daging (karnivora), keluarga felidae (kucing), genus panthera, dan tergolong dalam spesies tigris.\nharimau adalah jenis kucing terbesar dari spesiesnya, bahkan lebih besar dari singa. harimau juga adalah kucing tercepat kedua dalam berlari, setelah citah. dalam keseluruhan karnivora, harimau adalah kucing karnivora terbesar dan karnivora terbesar ketiga setelah beruang kutub dan beruang coklat.\nharimau biasanya memburu mangsa yang agak besar seperti rusa sambar, kijang, babi, atau kancil. namun, harimau akan memburu hewan kecil seperti landak apabila mangsa yang agak besar itu tidak ada. meskipun berasal dari keluarga yang sama, harimau berbeda dengan kucing biasa yang kecil, harimau sangat suka berenang, dan pada dasarnya kucing 